In [46]:
import geopandas as gpd
import json
from shapely.geometry import shape
from shapely.ops import nearest_points

# Функция для загрузки и обработки GeoJSON, обрабатывающая многокомпонентные геометрии
def load_and_split_geojson_safe(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)

    properties_list = []
    geometries_list = []

    for feature in data['features']:
        if 'geometry' not in feature or feature['geometry'] is None:
            continue
        geom = shape(feature['geometry'])
        if geom.type == 'MultiLineString':
            for line in geom.geoms:
                properties_list.append(feature['properties'])
                geometries_list.append(line)
        else:
            properties_list.append(feature['properties'])
            geometries_list.append(geom)

    gdf = gpd.GeoDataFrame(properties_list, geometry=geometries_list)
    return gdf

# Загрузка данных
lines_gdf_safe = load_and_split_geojson_safe(r"../short_lines.geojson")
points_gdf_safe = load_and_split_geojson_safe(r"../result_points.geojson")

# Установка системы координат (EPSG:4326), преобразование к метрической системе для точности (EPSG:32637)
lines_gdf_safe.crs = points_gdf_safe.crs = "EPSG:4326"
lines_gdf_safe = lines_gdf_safe.to_crs(epsg=4326)
points_gdf_safe = points_gdf_safe.to_crs(epsg=4326)

# Функция для поиска 5 ближайших точек к каждой линии
def find_nearest_points(line, points, num_points=5):
    # Сортировка точек по расстоянию до линии
    points['distance'] = points.geometry.apply(lambda point: line.geometry.distance(point))
    nearest_points = points.nsmallest(num_points, 'distance')
    return nearest_points

# Обработка каждой линии для определения 5 ближайших точек
for index, line in lines_gdf_safe.iterrows():
    nearby_points = find_nearest_points(line, points_gdf_safe)
    if nearby_points['defected'].any():
        lines_gdf_safe.at[index, 'defected'] = True
    else: 
        lines_gdf_safe.at[index, 'defected'] = False
    if not nearby_points.empty:
        most_common_class = nearby_points['class'].mode().iloc[0]
        lines_gdf_safe.at[index, 'most_common_class'] = most_common_class
    
    lines_gdf_safe.at[index, 'access_level']  = 'high'
    if lines_gdf_safe.at[index, 'most_common_class'] == 3:
        lines_gdf_safe.at[index, 'access_level']  = 'low'
    if lines_gdf_safe.at[index, 'most_common_class'] == 2:
        lines_gdf_safe.at[index, 'access_level']  = 'medium'
    if lines_gdf_safe.at[index, 'defected'] == True:
        lines_gdf_safe.at[index, 'access_level']  = 'low'

In [47]:
# Вывод результатов
lines_gdf_safe[lines_gdf_safe["defected"]==True].head(20)

,fid,geometry,defected,most_common_class,access_level
13,1.0,"LINESTRING (37.52778 55.73587, 37.52784 55.735...",True,3.0,low
16,1.0,"LINESTRING (37.52791 55.73588, 37.52788 55.73584)",True,3.0,low
17,1.0,"LINESTRING (37.52788 55.73584, 37.52787 55.735...",True,3.0,low
20,1.0,"LINESTRING (37.52797 55.73572, 37.52797 55.735...",True,3.0,low
21,1.0,"LINESTRING (37.52794 55.73568, 37.52791 55.735...",True,3.0,low
27,2.0,"LINESTRING (37.52779 55.73579, 37.52783 55.735...",True,3.0,low
28,2.0,"LINESTRING (37.52785 55.73582, 37.52787 55.73583)",True,3.0,low
32,4.0,"LINESTRING (37.52791 55.73565, 37.52794 55.735...",True,3.0,low
33,4.0,"LINESTRING (37.52795 55.73561, 37.52797 55.73560)",True,3.0,low
34,5.0,"LINESTRING (37.52797 55.73572, 37.52803 55.73569)",True,3.0,low


In [49]:
lines_gdf_safe.to_file("output.geojson", driver="GeoJSON")